# 사용자 정의 속성
- `Study` 객체는 `set_user_attr()`을 제공함
    - key는 `str`
    - value는 `json.dumps`로 직렬화할 수 있는 모든 개체이다

In [2]:
import sklearn.datasets
import sklearn.model_selection
import sklearn.svm

import optuna

study = optuna.create_study(storage = 'sqlite:///example.db')
study.set_user_attr('contributors', ['Akiba', 'Sano'])
study.set_user_attr('dataset', 'MNIST')

[I 2022-12-12 13:40:33,704] A new study created in RDB with name: no-name-18a9c442-397e-4ee8-a515-3e76523a9e98


- 명시한 속성엔 `user_attrs` 메서드로 접근함

In [3]:
study.user_attrs

{'contributors': ['Akiba', 'Sano'], 'dataset': 'MNIST'}

`StudySummary` 객체 : `get_all_study_summaries()`로 만들 수 있고, `study.user_attrs`와 동일

In [5]:
summaries = optuna.get_all_study_summaries('sqlite:///example.db')
summaries[0].user_attrs

{'contributors': ['Akiba', 'Sano'], 'dataset': 'MNIST'}

# Trial에 사용자 속성 추가하기

In [7]:
def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target
    
    svc_c = trial.suggest_float('svc_c', 1e-10, 1e10, log = True)
    clf = sklearn.svm.SVC(C = svc_c)
    
    # 사용자 속성 추가
    accuracy = sklearn.model_selection.cross_val_score(clf, x, y).mean()
    
    # 사용자 속성 정의
    trial.set_user_attr('accuracy', accuracy)
    
    return 1.0 - accuracy
    
study.optimize(objective, n_trials = 1)

[I 2022-12-12 13:43:43,807] Trial 0 finished with value: 0.05999999999999994 and parameters: {'svc_c': 3493585.896113314}. Best is trial 0 with value: 0.05999999999999994.


In [8]:
study.trials[0].user_attrs

{'accuracy': 0.9400000000000001}